In [1]:
import pandas as pd
from datetime import datetime
import requests
import numpy as np
from polygon.rest import RESTClient
from tqdm import tqdm
import warnings
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import json

warnings.simplefilter(action='ignore', category=FutureWarning)

api_key = 'hFrBS7nzcaLTa8mplO1ejm44DI4EscDM'
client = RESTClient(api_key)

In [2]:
def unix(date_str,hour,minute):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    #Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    #Create datetime object
    est_time = datetime(date.year, date.month, date.day, hour, minute, 0)
    #Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in milliseconds
    unix_timestamp_ms = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1000)
    return unix_timestamp_ms

<h1>Collect Trading Dates</h1>

In [3]:
def get_day(ticker,date_str):
    try:
        #get dataframe
        date = unix(date_str,9,30)
        data = client.get_aggs(ticker,
                                    multiplier=1,
                                    timespan='day',
                                    from_=date,
                                    to=date) 
        df = pd.DataFrame(data)
        end = df['open'].iloc[0]
    except Exception as e:
        #print(f"An exception error occurred for {ticker} on {date}: {e}")
        end = np.nan
    return end

In [4]:
#Get valid dates
all_bdays = pd.date_range('2020-06-01', '2024-09-30', freq='B').strftime('%Y-%m-%d').tolist()
dlist = []
for date in tqdm(all_bdays, desc="Validating Dates"):
    a = get_day('AAPL',date)
    if not np.isnan(a):
        dlist.append(date)
print(len(all_bdays),len(dlist))

Validating Dates: 100%|██████████| 1131/1131 [01:34<00:00, 11.91it/s]

1131 1091


In [6]:
#Set date range, save list for later use
print(f'{dlist[150]} - {dlist[-1]}')
use_dates = dlist[150:]
with open('dlist.json', 'w') as json_file:
    json.dump(dlist, json_file, indent=4)

2021-01-04 - 2024-09-30


<h1>Collect and Clean Tickers</h1>

In [8]:
def get_tickers(date):
    tickers = []
    types = []
    for type in ["CS","OS","ADRC"]:
        type_tickers = []
        url = 'https://api.polygon.io/v3/reference/tickers'
        params = {"market": "stocks","type": type,"date": date,"active": "true","limit": 1000,"apiKey": api_key}
        while url:
            response = requests.get(url, params=params)
            data = response.json()
            if 'results' in data:
                type_tickers.extend([item['ticker'] for item in data['results']])
            else:
                print(f"Error or no results for date {date}: {data}")
                break
            url = data.get('next_url', None)
        tickers.extend(type_tickers)
        types.extend([type] * len(type_tickers))
    return tickers,types,[date] * len(tickers)
#get_tickers('2024-10-24')

In [10]:
all_tickers = []
all_types = []
ticker_dates = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(get_tickers, use_dates), total=len(use_dates), desc="Processing tickers"))
    for tickers, types, dates in results:
        all_tickers.extend(tickers)
        all_types.extend(types)
        ticker_dates.extend(dates)

Processing tickers: 100%|██████████| 941/941 [07:02<00:00,  2.23it/s]


In [11]:
tickerdf = pd.DataFrame({'ticker':all_tickers,'type':all_types, 'date': ticker_dates})
tickerdf['key'] = tickerdf['ticker'] + '_' + tickerdf['date']
tickerdf = tickerdf.drop_duplicates(subset='key')
ticker_filt = tickerdf.groupby('ticker').agg(min_date=('date', 'min'),max_date=('date', 'max'),type=('type', 'first')).reset_index()
ticker_filt

,ticker,min_date,max_date,type
0,A,2021-01-04,2024-09-30,CS
1,AA,2021-01-04,2024-09-30,CS
2,AAC,2021-01-04,2024-05-17,CS
3,AACG,2021-01-04,2024-09-30,ADRC
4,AACI,2021-11-10,2024-08-15,CS
...,...,...,...,...
9384,ZX,2021-01-04,2024-05-17,CS
9385,ZY,2021-04-22,2022-10-19,CS
9386,ZYME,2021-01-04,2024-09-30,CS
9387,ZYNE,2021-01-04,2023-10-11,CS


In [13]:
def primary_listing(ticker,start_date,end_date):
    drange = pd.bdate_range(start=start_date, end=end_date).strftime('%Y-%m-%d').tolist()
    for date in drange:
        response = requests.get(f'https://api.polygon.io/v3/reference/tickers/{ticker}?date={date}&apiKey={api_key}')
        data = response.json()
        if data['status'] == 'OK':
            if (data['results']['market'] != 'otc') & ('ticker_root' in data['results']):
                return data['results']['ticker_root']
    return 'fail'
#primary_listing('AAPL','2023-11-07','2024-11-07')

In [16]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    roots = list(tqdm(executor.map(primary_listing, ticker_filt['ticker'], ticker_filt['min_date'], ticker_filt['max_date']), total=len(ticker_filt), desc="Processing tickers"))
ticker_filt['root'] = roots

Processing tickers: 100%|██████████| 9389/9389 [04:23<00:00, 35.65it/s]


In [17]:
finaldf = ticker_filt[ticker_filt['root'] == ticker_filt['ticker']]
finaldf.to_csv('base.csv')
finaldf

,ticker,min_date,max_date,type,root
0,A,2021-01-04,2024-09-30,CS,A
1,AA,2021-01-04,2024-09-30,CS,AA
2,AAC,2021-01-04,2024-05-17,CS,AAC
3,AACG,2021-01-04,2024-09-30,ADRC,AACG
4,AACI,2021-11-10,2024-08-15,CS,AACI
...,...,...,...,...,...
9384,ZX,2021-01-04,2024-05-17,CS,ZX
9385,ZY,2021-04-22,2022-10-19,CS,ZY
9386,ZYME,2021-01-04,2024-09-30,CS,ZYME
9387,ZYNE,2021-01-04,2023-10-11,CS,ZYNE
